# `data` namespace

**This is a proof-of-concept**

## Classes

<div style="background-color:#ffffee;border:1px solid #ddddaa;margin:2em;padding:5px;">

### `data.Series` class

An instance of the `data.Series` class contains a labelled array.

Bracket indexing of the series gives access to the values of the array.
The **`values`** property is equivalent to `[]`.
The label, which can take any value,
can be accessed through the **`label`** property.

</div>
<div style="background-color:#ffffee;border:1px solid #ddddaa;margin:2em;padding:5px;">

### `data.Frame` class

An instance of the `data.Frame` class contains a list of `data.Series` instances.
All the series must contain values arrays of the same length.

The series list can be accessed by bracket indexing of rank 1 using the labels of the series as indices.
Bracket indexing of rank 2 gives access to the values in the series.
The properties **`series`**, **`labels`** and **`values`** are equivalent to `[]`, `[].label` and `[;]`.

Frames are displayed with shades at row intervals of the size specified by the **SHADE** property
up to a maximum number of lines specified by the **MAXLINES** property.

</div>

## Functions and operators

<div style="background-color:#eeffee;border:1px solid #aaddaa;margin:2em;padding:5px;">

### `data.series` function

This function returns an instance or a list of instances of the `data.Series` class.

- `⍺ data.series ⍵` creates an instance of `data.Series` with label `⍺` and values `⍵`. If `⍺` is a series, the label is taken from it.
- `data.series ⍵` creates an instance of `data.Series` for each of the series in `⍵` and each of the series contained in each frame in `⍵`. If `⍵` is a rank 2 array, it must contain series with the same label in each column, and their values will be concatenated.

</div>
<div style="background-color:#eeffee;border:1px solid #aaddaa;margin:2em;padding:5px;">

### `data.frame` function

This function returns an instance of the `data.Frame` class.

- `⍺ data.frame ⍵` creates an instance of `data.Frame` with labels `⍺` (or the labels of the series list or frame `⍺`) and values `⍵`. If `⍵` is a string, it writes the frame `⍺` to the CSV file `⍵` or reads the CSV file `⍵` without header and returns frame with labels `⍺`.
- `data.frame ⍵` creates an instance of `data.Frame` with each of the series returned by `data.series ⍵`. If `⍵` is a string, it reads the file `⍵` as CSV with header and returns frame.

</div>
<div style="background-color:#ffeeff;border:1px solid #ddaadd;margin:2em;padding:5px;">

### `data.sort` operator

This operator sorts data according to the left function.

- `⍺ (⍺⍺ data.sort) ⍵` returns `⍵` (a frame, list of series, or array) sorted according to the result of `⍺⍺ ⍺` (where `⍺⍺` typically is one of `⍒⍋`).
- `(⍺⍺ data.sort) ⍵` is equivalent to `(⍺⍺ data.sort)⍨⍵`.

</div>
<div style="background-color:#ffeeff;border:1px solid #ddaadd;margin:2em;padding:5px;">

### `data.by` operator

This operator groups data by the right operand and applies the left function.

- `⍺ (⍺⍺ data.by ⍵⍵) ⍵` returns the data in `⍵` (a frame or list of series) grouped according to `⍵⍵` (also a frame or list of series) and apply `⍺⍺` to each group. If `⍺⍺` and `⍵⍵` are both series or label values, the values in `⍺⍺` are grouped for each value in `⍵⍵` and distributed in series, and a new frame is returned. The labels of the new frame (either all of them, the ones not in `⍵`, or the ones not in `⍵⍵`) are given in `⍺`, which can be a list of values, a list of series, or a frame.
- `(⍺⍺ data.by ⍵⍵) ⍵` is equivalent to `(⍺⍺ data.by ⍵⍵)⍨⍵`. If `⍺⍺` and `⍵⍵` are both series or label values, the values in `⍺⍺` are grouped for each value in `⍵⍵` and distributed in series, and a new frame is returned.

</div>
<div style="background-color:#ffeeff;border:1px solid #ddaadd;margin:2em;padding:5px;">

### `data.join` operator

This operator merges two frames (or lists of series).

- `⍺ (⍺⍺ data.join ⍵⍵) ⍵` returns frame with series labelled `⍺[].label ⍵⍵ ⍵.labels`. If two series at left and right have the same label, its values are combined as `⍺.values ⍺⍺ ⍵.values`.
- `(⍺⍺ data.join ⍵⍵) ⍵` returns a series with label `⍵⍵ ⍵.labels` and values `⍺⍺ ⍵.values`.

</div>

## Examples

In [1]:
)clear
⎕PP←3
] _←link.import # .  ⍝ import data namespace from current directory

clear ws

### Berkeley

In [2]:
f ←   data.frame'berkeley.csv'
a ←   data.('Applicants' 'Accepted'{(≢⍵),('A'+.=⊃¨)⍵}by'Major' 'Gender'⊢)f[]~f[⊂'Year']
g ← a data.(⍋sort⊣,join⊣(⊂'Gender')(+⌿,'T'⍨)by(⊂'Major')⊢)a[]~a[⊂'Gender']
m ← g data.(⍋sort⊣,join⊣(⊂'Major')(+⌿,(⊂'Total')⍨)by(⊂'Gender')⊢)g[]~g[⊂'Major']
r ← m data.(frame⊣,'%Accepted'series⊢)100×÷/m[;'Accepted' 'Applicants']
    r data.(frame⊣,'%Applicants'series⊢)(100×⊢÷≢⍴('T'=⊃¨r[;⊂'Major'])⌿⊢)⊢r[;⊂'Applicants']

Major Gender Applicants Accepted %Accepted %Applicants
─────────────────────────────────────────────────────────────
 A F 108 89 82.4 2.5
 M 1138 825 72.5 13.5
░░░░░░░░T░░░░░░░░░░░░░1246░░░░░░░914░░░░░░73.4░░░░░░░░░9.76░░
 B F 25 17 68 0.579
 M 560 353 63 6.63
░░░░░░░░T░░░░░░░░░░░░░░585░░░░░░░370░░░░░░63.2░░░░░░░░░4.58░░
 C F 593 201 33.9 13.7
 M 325 120 36.9 3.85
░░░░░░░░T░░░░░░░░░░░░░░918░░░░░░░321░░░░░░35░░░░░░░░░░░7.19░░
 D F 375 131 34.9 8.68
 M 417 138 33.1 4.94
░░░░░░░░T░░░░░░░░░░░░░░792░░░░░░░269░░░░░░34░░░░░░░░░░░6.21░░
 E F 393 94 23.9 9.1
 M 191 53 27.7 2.26
░░░░░░░░T░░░░░░░░░░░░░░584░░░░░░░147░░░░░░25.2░░░░░░░░░4.58░░
 F F 341 25 7.33 7.89
 M 373 22 5.9 4.42
░░░░░░░░T░░░░░░░░░░░░░░714░░░░░░░░47░░░░░░░6.58░░░░░░░░5.59░░
 Other F 2486 937 37.7 57.5
 M 5438 2227 41 64.4
░░░░░░░░T░░░░░░░░░░░░░7924░░░░░░3164░░░░░░39.9░░░░░░░░62.1░░░
 Total F 4321 1494 34.6 100
 M 8442 3738 44.3
░░░░░░░░T░░░░░░░░░░░░12763░░░░░░5232░░░░░░41░░░░░░░░░░░░░░░░░

### Iris

In [3]:
AVG←+⌿÷≢ ⋄ STD←(2*∘÷⍨+⌿÷¯1+≢)2*⍨⊢-⍤1+⌿÷≢     ⍝ average and standard deviation
PCT←{((2÷⍨+/)⊢⌷⍨∘⊂⍋⌷⍨∘⊂∘⌈100÷⍨⍺×0 1+≢)⍵}     ⍝ percentile-⍺
PCC←+.×⍥((⊢÷2*∘÷⍨+.×⍨)⊢-+⌿÷≢)                ⍝ Pearson correlation coefficient
f←'sl' 'sw' 'pl' 'pw' 'class'data.frame'iris.csv'
aggs←{l f←⍺⍵ ⋄ ⍪,¨⍺⍺{(⍺⍺ ⍵)⍵⍵data.by(f[l])⊢f[⊂⍵]}⍵⍵¨f.labels~l}
s←(⊂'class'){'⌊AS⌈',¨⊂⍵}aggs(⌊⌿,AVG,STD,⌈⌿)f
p←(⊂'class')('25' '50' '75',⍨¨⊂)aggs(,25 50 75∘.PCT↓∘⍉)f
s←s{data.frame⍺.series,1↓⍵.series}¨¨p ⋄ s.SHADE←0
pcc←{∘.PCC⍨↓⍉⍵}data.by(⊂'class')⊢f[]
pcc.values⍪←(⊂'Class'),({(∪⍵)⍳⍵}f[;⊂'class'])∘PCC¨f[f.labels~⊂'class'].values
⍕(⍪⊃(⊣,(⊂''),⊢)⌿s)pcc

class ⌊sl Asl Ssl ⌈sl sl25 sl50 sl75 class sl sw pl pw 
 ────────────────────────────────────────────────────────── ────────────────────────────────────────────── 
 Iris-setosa 4.3 5.01 0.352 5.8 4.8 5 5.2 Iris-setosa 1 0.747 0.264 0.279 
 Iris-versicolor 4.9 5.94 0.516 7 5.6 5.9 6.3 0.747 1 0.177 0.28 
 Iris-virginica 6.59 0.636 7.9 6.2 6.5 7 ░░░░░░░░░░░░░░░░░░0.264░░░0.177░░1░░░░░░0.306░ 
 0.279 0.28 0.306 1 
 class ⌊sw Asw Ssw ⌈sw sw25 sw50 sw75 Iris-versicolor 1 0.526 0.754 0.546 
 ────────────────────────────────────────────────────────── ░░░░░░░░░░░░░░░░░░0.526░░░1░░░░░░0.561░░0.664░ 
 Iris-setosa 2.3 3.42 0.381 4.4 3.1 3.4 3.7 0.754 0.561 1 0.787 
 Iris-versicolor 2 2.77 0.314 3.4 2.5 2.8 3 0.546 0.664 0.787 1 
 Iris-virginica 2.2 2.97 0.322 3.8 2.8 3 3.2 ░Iris-virginica░░░1░░░░░░░0.457░░0.864░░0.281░ 
 0.457 1 0.401 0.538 
 class ⌊pl Apl Spl ⌈pl pl25 pl50 pl75 0.864 0.401 1 0.322 
 ────────────────────────────────────────────────────────── ░░░░░░░░░░░░░░░░░░0.281░░░0.538░░0.322░░1░░░░░ 
 Iris-setosa 1 1.46 0.174 1.9 1.4 1.5 1.6 Class 0.783 ¯0.419 0.949 0.956 
 Iris-versicolor 3 4.26 0.47 5.1 4 4.35 4.6 
 Iris-virginica 4.5 5.55 0.552 6.9 5.1 5.55 5.9 
 
 class ⌊pw Apw Spw ⌈pw pw25 pw50 pw75 
 ─────────────────────────────────────────────────────────── 
 Iris-setosa 0.1 0.244 0.107 0.6 0.2 0.2 0.3 
 Iris-versicolor 1 1.33 0.198 1.8 1.2 1.3 1.5 
 Iris-virginica 1.4 2.03 0.275 2.5 1.8 2 2.3

### Google

In [4]:
g←'date' 'n'data.frame⎕CSV⍠'Invert'2⊢(3↓⊃⎕NGET'google-scotch.csv'1)'N'4
d←{⍲/(∧/∊∘(⎕D,'-'))¨⍵:⎕SIGNAL 11 ⋄ ↑'-'(⍎¨≠⊆⊢)¨⍵}g[;⊂'date']
tt←'n'data.by'month'⊢t←'year' 'month' 'n'+⌿data.by(d[;1 2])⊢g[⊂'n']
tt←(tt.labels,⊂'total')data.frame(tt[;,⊂'year']⍪⊂'total'),(⊢⍪(+⌿+/¨))(⊢,(+/+/¨))tt[;1↓tt.labels]
c←'n'data.by'month'⊢t data.frame t[;t.labels~'n'],(⊂⍬),¯2-/t[;'n']
tt data.frame tt[;]⍪(⊂'')⍪c[;],(⊂⍬)⍪¯2-⌿¯1↓tt[;,⊂'total']
{'min' 'avg' 'max' 'total'(⌊⌿,(+⌿÷≢),⌈⌿,+⌿)data.by(t[⊂⍵])⊢t[⊂'n']}¨'year' 'month'

year 1 2 3 4 5 6 7 8 9 10 11 12 total
──────────────────────────────────────────────────────────────────────────────────────
 2019 · · · · · · · 198 334 275 296 439 1542
 2020 288 272 301 291 361 279 275 336 264 253 366 350 3636
░░2021░░░332░░░264░░░250░░░245░░░317░░░247░░░249░░░306░░░242░░░315░░░271░░░320░░░3358░
 2022 331 252 249 256 314 250 315 253 237 303 256 303 3319
 2023 321 234 247 301 235 238 306 240 230 289 250 352 3243
░░2024░░░238░░░227░░░293░░░239░░░231░░░286░░░229░░░111░░░░░·░░░░░·░░░░░·░░░░░·░░░1854░
 total 1510 1249 1340 1332 1458 1300 1374 1444 1307 1435 1439 1764 16952

░░2019░░░░░·░░░░░·░░░░░·░░░░░·░░░░░·░░░░░·░░░░░·░░░░░·░░░136░░░¯59░░░░21░░░143░░░░░░·░
 2020 ¯151 ¯16 29 ¯10 70 ¯82 ¯4 61 ¯72 ¯11 113 ¯16 2094
 2021 ¯18 ¯68 ¯14 ¯5 72 ¯70 2 57 ¯64 73 ¯44 49 ¯278
░░2022░░░░11░░░¯79░░░░¯3░░░░░7░░░░58░░░¯64░░░░65░░░¯62░░░¯16░░░░66░░░¯47░░░░47░░░░¯39░
 2023 18 ¯87 13 54 ¯66 3 68 ¯66 ¯10 59 ¯39 102 ¯76
 2024 ¯114 ¯11 66 ¯54 ¯8 55 ¯57 ¯118 · · · · ¯1389

year min avg max total month min avg max total
 ──────────────────────────── ─────────────────────────────
 2019 198 308 439 1542 1 238 302 332 1510
 2020 253 303 366 3636 2 227 250 272 1249
 2021░░░242░░280░░332░░3358░░ ░░░░░3░░247░░268░░301░░░1340░
 2022 237 277 331 3319 4 239 266 1332
 2023 230 270 352 3243 5 231 292 361 1458
 2024░░░111░░232░░293░░1854░░ ░░░░░6░░238░░260░░286░░░1300░
 7 229 275 315 1374
 8 111 241 336 1444
 ░░░░░9░░230░░261░░334░░░1307░
 10 253 287 315 1435
 11 250 288 366 1439
 ░░░░12░░303░░353░░439░░░1764░

    jgl@dyalog.com